### Sentiment Analyse 
#### Ausgangslage
In einem Online Shop können Produkte Reviews in Form von Text geschrieben werden. Wir möchten herausfinden ob der Text positiv oder negativ ist und so die Bewertung für das Produkt erstellen zu können.

#### Aufgabe
Anhand von Produkten von Amazon wollen wir den Sentiment für die Produkte berechnen. Dies tun wir mittels Textanalyse und Supervised Learning Algorithmen wie MaxEnt oder Naive Bayes.

In [ ]:
# Installation der NLTK Library
!pip install nltk

In [ ]:
%matplotlib inline
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk

import warnings
warnings.filterwarnings('ignore')

#### Laden der Produkt Daten

In [ ]:
csv_data = pd.read_csv(r"1429_1.csv")
csv_data.head()

#### Daten für die Sentiment Analyse extrahieren und null values analyse

In [ ]:
data = csv_data[['reviews.rating' , 'reviews.text' , 'reviews.title' , 'reviews.username']]
print(data.isnull().sum())
data.head()

#### Sentiment anhand des Ratings setzen
Alle Produkte die ein Rating von 4 und grösser haben sind positiv. Alle Produkte mit einem Rating von Rating sind neutral und alle die kleiner als 3 sind sind negativ. Das DataFrame mit einer neuen Spalte ergänzen die den Wert positiv oder negativ hat.

In [ ]:
def set_rating_sentiment(rating):
    if rating >= 4:
        return "positive"
    elif rating == 3:
        return "neutral"
    else:
        return "negative"

data["sentiment_rating"] = data["reviews.rating"].apply(set_rating_sentiment)
data["sentiment_rating"].value_counts().plot.bar()
data["sentiment_rating"].value_counts()

#### TODO: Text für die Sentiment Analyse vorbereiten
Cleanup des Textes implementieren.

In [ ]:
# TODO: Methode cleanup_text implementieren (Array mit allen Worten in lowercase erstellen und Leerzeichen entfernen)
cleanup_re = re.compile('[^a-z]+')
def cleanup_text(sentence):
    # TODO
    return sentence

#### Cleanup des Text für alle Rows durchführen

In [ ]:
sentiment = data[data["reviews.text"].notnull()]
sentiment["text_clean"] = sentiment["reviews.text"].apply(cleanup_text)
sentiment.head()

#### Implementation der Sentiment Analyse mittels NLTK
Implementierung einer einfachen Sentiment Analyse des Textes. Das Sentiment muss mit einer Spalte mit dem dem Text Sentiment ergänzt werden. Dies kann mittels SentimentIntensityAnalyzer https://www.nltk.org/api/nltk.sentiment.html umgesetzt werden. Für den Score des Textes kann die Methode polarity_scores verwendet werden. Ist der Score 0.0 ist der Sentiment neutral, ist der Score grösser als 0.0 ist der Sentiment positiv ansonsten ist er negativ. 

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sentiment_analyzer = SentimentIntensityAnalyzer()

In [ ]:
# TODO: Methode get_sentiment(text) implementieren (Entsprechend 'neutral', 'positive' oder 'negativ' zurückgeben)
def get_sentiment(text):
    # TODO

In [ ]:
sentiment["sentiment_text"] = sentiment["text_clean"].apply(get_sentiment)

sentiment["sentiment_text"].value_counts().plot.bar()
sentiment["sentiment_text"].value_counts()

#### Ausgabe der negativen Kommentare

In [ ]:
negative_summary = list()
for k, v in sentiment.iterrows():
    if v["sentiment_text"] == "negative":
        negative_summary.append({"text": v["text_clean"], "sentiment": v["sentiment_text"]})

pd.set_option('display.max_colwidth', -1)      
pd.DataFrame(negative_summary).head(50)

#### Vergleich Rating und Text Sentiment

In [ ]:
sentiment["equal"] = sentiment["sentiment_rating"] == sentiment["sentiment_text"]

sentiment["equal"].value_counts().plot.bar()
sentiment["equal"].value_counts()

#### Sentiment Analyse mittls Naive Bayes Classifier
Die Daten werden für den Classifier aufbereitet

In [ ]:
sentiment["sentiment_classifier"] = sentiment["reviews.rating"] >= 4
sentiment["sentiment_classifier"] = sentiment["sentiment_classifier"].replace([True , False] , ["pos" , "neg"])

In [ ]:
sentiment_classifier = sentiment[["reviews.text", "sentiment_classifier"]][:100].values
all_words = set(word.lower() for s in sentiment_classifier for word in nltk.tokenize.word_tokenize(s[0]))
train = [({word: (word in nltk.tokenize.word_tokenize(x[0])) for word in all_words}, x[1]) for x in sentiment_classifier]

#### TODO: Naive Bayes Classifier trainieren
Der Classifier muss zuerst trainiert werden. Traniert den NaiveBayesClassifier https://www.nltk.org/book/ch06.html aus der NLTK Libary mit den Trainingsdaten.

In [ ]:
# TODO: NaiveBayesClassifier trainieren

#### TODO: Satz mit dem trainierten Classifier klassifizieren
Der oben trainierte Classifieren mittels eines Satz testen. Der Wert der zurückgeliefert wird ist "pos" oder "neg". Der Satz muss zuerst in Feaures umgewandelt werden.

In [ ]:
def sentence_to_feature(sentence):
    return {word.lower(): (word in nltk.tokenize.word_tokenize(test_sentence.lower())) for word in all_words}

In [ ]:
# TODO: Irgeneinen Satz in Englisch testen mit sentence_to_feature(test_sentence)

#### TODO: Bestimmen der Accuracy
Zum Schluss noch die Metrik Accuracy für den trainierten Classifier bestimmen.
* Erstellen eines Test Datasets mit den Rows von 101 bis 200
* Accuracy für das Test Dataset bestimmen

In [ ]:
sentiment_classifier = sentiment[["reviews.text", "sentiment_classifier"]][101:200].values
all_words_test = set(word.lower() for s in sentiment_classifier for word in nltk.tokenize.word_tokenize(s[0]))
test = [({word: (word in nltk.tokenize.word_tokenize(x[0])) for word in all_words_test}, x[1]) for x in sentiment_classifier]

# TODO: Accuracy testen mit nltk-classifier